In [48]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.nn import softmax
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.losses import categorical_crossentropy

In [53]:
df = pd.read_csv("./weather.csv")

# deleting bad data
df.drop(["Daily Summary","Formatted Date","Loud Cover","Apparent Temperature (C)"], axis=1, inplace = True)
df.dropna(axis=0,inplace=True)

df["Summary"].value_counts()


Partly Cloudy                          31635
Mostly Cloudy                          27914
Overcast                               16516
Clear                                  10763
Foggy                                   7117
Breezy and Overcast                      528
Breezy and Mostly Cloudy                 516
Breezy and Partly Cloudy                 386
Dry and Partly Cloudy                     86
Windy and Partly Cloudy                   67
Light Rain                                63
Breezy                                    54
Windy and Overcast                        45
Humid and Mostly Cloudy                   40
Drizzle                                   39
Breezy and Foggy                          35
Windy and Mostly Cloudy                   35
Dry                                       34
Humid and Partly Cloudy                   17
Dry and Mostly Cloudy                     14
Rain                                      10
Windy                                      8
Humid and 

In [54]:

# making label's frequecy same, to avoid bias
for summary,s_df in df.groupby("Summary"):
    if len(s_df)< 5000:
        df.drop(s_df.index,axis=0,inplace=True)

    elif len(s_df) > 7000:
        x = len(s_df) - 7000
        df.drop(s_df[:x].index,axis=0,inplace=True)
        
df.reset_index(inplace=True)

# converting categorical data into numeric
df[["rain","snow"]] = pd.get_dummies(df["Precip Type"])
df.drop(["Precip Type","index"], axis = 1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Summary                 35000 non-null  object 
 1   Temperature (C)         35000 non-null  float64
 2   Humidity                35000 non-null  float64
 3   Wind Speed (km/h)       35000 non-null  float64
 4   Wind Bearing (degrees)  35000 non-null  int64  
 5   Visibility (km)         35000 non-null  float64
 6   Pressure (millibars)    35000 non-null  float64
 7   rain                    35000 non-null  uint8  
 8   snow                    35000 non-null  uint8  
dtypes: float64(5), int64(1), object(1), uint8(2)
memory usage: 1.9+ MB


In [5]:
# normalization
x = df[["Humidity","Pressure (millibars)", "Temperature (C)","Visibility (km)","Wind Bearing (degrees)","Wind Speed (km/h)"]]
org_mean, org_std = x.mean(), x.std()

df[["Humidity","Pressure (millibars)", "Temperature (C)","Visibility (km)","Wind Bearing (degrees)","Wind Speed (km/h)"]] = (x - org_mean)/ org_std


targets = df[["Summary"]]
inputs = df.drop(["Summary"], axis= 1)
inputs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47117 entries, 0 to 47116
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (C)         47117 non-null  float64
 1   Humidity                47117 non-null  float64
 2   Wind Speed (km/h)       47117 non-null  float64
 3   Wind Bearing (degrees)  47117 non-null  float64
 4   Visibility (km)         47117 non-null  float64
 5   Pressure (millibars)    47117 non-null  float64
 6   rain                    47117 non-null  uint8  
 7   snow                    47117 non-null  uint8  
dtypes: float64(6), uint8(2)
memory usage: 2.2 MB


In [6]:
# one hot encoding the targets 
one_hot_targets = pd.get_dummies(targets)

labels = targets["Summary"].unique()
labels.sort()
labels

array(['Clear', 'Foggy', 'Mostly Cloudy', 'Overcast', 'Partly Cloudy'],
      dtype=object)

In [71]:
# converting into tensors
inputs_tf = tf.constant(inputs)
targets_tf = tf.cast(tf.constant(one_hot_targets), tf.float64)

In [70]:
model = Sequential([
    layers.Dense(5),
    layers.Softmax()
])

model.build(input_shape=inputs_tf.shape)

In [68]:
model.compile(
    loss = categorical_crossentropy,
    optimizer = SGD(learning_rate=0.001),
    metrics = ["accuracy"]
)

In [69]:
history = model.fit(
    inputs_tf,
    targets_tf,
    epochs = 100,
    batch_size = 70
)

Epoch 1/100
674/674 [==============================] - 1s 907us/step - loss: 1.8188 - accuracy: 0.2192
Epoch 2/100
674/674 [==============================] - 1s 900us/step - loss: 1.6444 - accuracy: 0.2344
Epoch 3/100
674/674 [==============================] - 1s 963us/step - loss: 1.5606 - accuracy: 0.2777
Epoch 4/100
674/674 [==============================] - 1s 1ms/step - loss: 1.5051 - accuracy: 0.3346
Epoch 5/100
674/674 [==============================] - 1s 966us/step - loss: 1.4614 - accuracy: 0.3785
Epoch 6/100
674/674 [==============================] - 1s 953us/step - loss: 1.4239 - accuracy: 0.4069
Epoch 7/100
674/674 [==============================] - 1s 957us/step - loss: 1.3909 - accuracy: 0.4242
Epoch 8/100
674/674 [==============================] - 1s 884us/step - loss: 1.3620 - accuracy: 0.4337
Epoch 9/100
674/674 [==============================] - 1s 921us/step - loss: 1.3368 - accuracy: 0.4400
Epoch 10/100
674/674 [==============================] - 1s 890us/step - los

In [64]:
history_df = pd.DataFrame(history.history)
history_df

,loss,accuracy
0,1.104360,0.505911
1,1.104245,0.506059
2,1.104134,0.505953
3,1.104022,0.506102
4,1.103912,0.506081
5,1.103803,0.506144
6,1.103693,0.506208
7,1.103587,0.506017
8,1.103478,0.506081
9,1.103371,0.506293


In [ ]:
# initializing the model
weights = tf.Variable(tf.random.normal(shape=(8,5), dtype = tf.float64))
bias = tf.Variable(tf.random.normal(shape=(1,5), dtype = tf.float64))

In [ ]:
# main training loop
lr = 0.01
epochs = range(500)
for epoch in tqdm(epochs):
    
    with tf.GradientTape() as tape:
        
        # making prediction
        preds = tf.add(inputs_tf @ weights, bias)
        prob = softmax(preds)
        
        # calculating loss
        loss = categorical_crossentropy(targets_tf, prob)
        mean_loss = tf.reduce_sum(loss)/ len(targets_tf)
        
        # print(epoch, mean_loss)
        
        # taking gradints
        w_grad, b_grad = tape.gradient( mean_loss , [weights,bias] )
        
        
        # updating model
        weights = tf.Variable(weights - lr * w_grad)
        bias = tf.Variable(bias - lr * b_grad)

In [ ]:
def predict(input_):
    pred = input_ @ weights + bias
    prob = softmax(pred)

    prob = list(prob[0])
    
    for label, p in zip(labels, prob):
        print(f"{label} : {float(p)*100:.2f}%")
        
    return labels[int(tf.where(pred[0] == tf.reduce_max(pred[0]))[0,0])]
    
    
predict(inputs.iloc[100]), targets.iloc[100]

In [125]:
input_ = np.array([[30, 0.5, 10, 90, 10, 1000, 1, 0]])

temp = (30 - org_mean["Temperature (C)"])/ org_std["Temperature (C)"]
humid = (0.5 - org_mean["Humidity "])
# predict(input_)

2.018313361669085